1- Installation And Download

In [1]:
!pip -q install pyxlsb openpyxl

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 200)


In [2]:
from google.colab import files
uploaded = files.upload()
file_path = next(iter(uploaded))


Saving stc TV Data Set_T1.xlsb to stc TV Data Set_T1.xlsb


In [3]:
engine = "pyxlsb" if file_path.lower().endswith(".xlsb") else None

xls = pd.ExcelFile(file_path, engine=engine)
print("Sheets:", xls.sheet_names)

# اختر الشيت المناسب؛ إن وُجد Final_Dataset نستخدمه، وإلا أول شيت
sheet = "Final_Dataset" if "Final_Dataset" in xls.sheet_names else xls.sheet_names[0]

df = pd.read_excel(file_path, sheet_name=sheet, engine=engine)
print("Loaded:", file_path, "| Sheet:", sheet)
df.head(3)


Sheets: ['Final_Dataset']
Loaded: stc TV Data Set_T1.xlsb | Sheet: Final_Dataset


,Column1,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name
0,1,42882,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets
1,3,42876,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana
2,4,42957,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess


2- Checking the data format

In [4]:
#To ensure the size and types of columns before any cleaning.print("Shape (rows, cols):", df.shape)
print("\nColumn dtypes:")
print(df.dtypes)

print("\nPreview:")
display(df.head(10))



Column dtypes:
Column1              int64
date_                int64
user_id_maped        int64
program_name        object
duration_seconds     int64
program_class       object
season               int64
episode              int64
program_desc        object
program_genre       object
series_title         int64
hd                   int64
original_name       object
dtype: object

Preview:


,Column1,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name
0,1,42882,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets
1,3,42876,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana
2,4,42957,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
3,5,42942,19307,The Mermaid Princess,76,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
4,7,42923,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,1,Churchill
5,8,42966,20775,Beavis And Butt-Head Do America,3,MOVIE,0,0,Animation MovieBeavis And Butt-Head Do America,Animation,0,0,Beavis And Butt-Head Do America
6,9,42912,11374,The Mermaid Princess,33577,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess
7,10,43188,6358,Coco,14,MOVIE,0,0,Animation MovieCoco (HD),Animation,0,1,Coco
8,11,43127,11660,Kidnap,85,MOVIE,0,0,Action MovieKidnap (HD),Action,0,1,Kidnap
9,12,42824,5155,The Accountant,42,MOVIE,0,0,Action MovieThe Accountant (HD),Action,0,1,The Accountant


3- Initial cleaning and basic transformations


In [5]:
import numpy as np
import pandas as pd

df1 = df.copy()

# 3.1) Delete the row numbering column if it’s just a sequence
if "Column1" in df1.columns:
    if df1["Column1"].is_monotonic_increasing and df1["Column1"].iloc[0] in (0,1):
        df1 = df1.drop(columns=["Column1"])

# 3.2) Clean the text columns
str_cols = ["program_name","program_desc","program_genre","original_name","program_class"]
for c in str_cols:
    if c in df1.columns:
        df1[c] = df1[c].astype(str).str.strip()
        df1[c] = df1[c].replace({"nan": np.nan, "None": np.nan, "": np.nan})

# 3.3) Convert the date from date_ (int) to smart datetime
def parse_date_int(x: int):
    try:
        x = int(x)
    except:
        return pd.NaT
    # yyyymmdd
    if 19000101 <= x <= 21001231:
        return pd.to_datetime(str(x), format="%Y%m%d", errors="coerce")
    # Excel serial (≈ dates from 1900)
    if 20000 < x < 60000:
        return pd.to_datetime("1899-12-30") + pd.to_timedelta(x, unit="D")
    # Unix epoch seconds
    if x > 10**9 and x < 10**12:
        return pd.to_datetime(x, unit="s", errors="coerce")
    # Unix epoch millis
    if x >= 10**12:
        return pd.to_datetime(x, unit="ms", errors="coerce")
    return pd.NaT

df1["watch_date"] = df1["date_"].apply(parse_date_int)

# 3.4) Viewing minutes from seconds
if "duration_seconds" in df1.columns:
    df1["duration_min"] = (df1["duration_seconds"] / 60).round(2)

# 3.5) Quality reading column (HD/SD) from 0/1
if "hd" in df1.columns:
    df1["quality"] = np.where(df1["hd"]==1, "HD", "SD")

# Display a quick summary after cleaning
print("Shape after basic prep:", df1.shape)
df1.head(3)


Shape after basic prep: (1048575, 15)


,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name,watch_date,duration_min,quality
0,42882,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets,2017-05-27,0.67,SD
1,42876,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana,2017-05-21,0.28,HD
2,42957,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess,2017-08-10,0.13,HD


4- Checking for missing values and duplicates

In [6]:
# 4.1) Missing values for each column
missing = df1.isna().sum().to_frame("missing_count")
missing["missing_pct"] = (missing["missing_count"]/len(df1)*100).round(2)
missing.sort_values("missing_pct", ascending=False)


,missing_count,missing_pct
program_desc,14038,1.34
user_id_maped,0,0.00
date_,0,0.00
duration_seconds,0,0.00
program_class,0,0.00
season,0,0.00
program_name,0,0.00
episode,0,0.00
program_genre,0,0.00
series_title,0,0.00


In [7]:
# 4.2) Duplicate rows
dup_rows = int(df1.duplicated().sum())
dup_rows


102074

5- Create a cleaned copy with simple imputation

In [9]:
df_clean = df1.drop_duplicates().copy()

# Illogical values -> NaN (then impute)
if "duration_seconds" in df_clean.columns:
    df_clean.loc[df_clean["duration_seconds"] <= 0, "duration_seconds"] = np.nan
if "duration_min" in df_clean.columns:
    df_clean.loc[df_clean["duration_min"] <= 0, "duration_min"] = np.nan

for col in ["season","episode"]:
    if col in df_clean.columns:
        df_clean.loc[df_clean[col] <= 0, col] = np.nan
        # Use the Int64 type that supports NaN
        df_clean[col] = df_clean[col].astype("Int64")

# Impute missing values
for col in df_clean.columns:
    if pd.api.types.is_numeric_dtype(df_clean[col]):
        if df_clean[col].isna().any():
            df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    else:
        if df_clean[col].isna().any():
            mode = df_clean[col].mode(dropna=True)
            df_clean[col] = df_clean[col].fillna(mode.iloc[0] if not mode.empty else "Unknown")

print("Missing BEFORE:", int(df1.isna().sum().sum()))
print("Missing AFTER :", int(df_clean.isna().sum().sum()))
df_clean.head(3)


Missing BEFORE: 14038
Missing AFTER : 0


,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name,watch_date,duration_min,quality
0,42882,26138,100 treets,40.0,MOVIE,1,9,Drama Movie100 Streets,Drama,0,0,100 treets,2017-05-27,0.67,SD
1,42876,7946,Moana,17.0,MOVIE,1,9,Animation MovieMoana (HD),Animation,0,1,Moana,2017-05-21,0.28,HD
2,42957,7418,The Mermaid Princess,8.0,MOVIE,1,9,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess,2017-08-10,0.13,HD


6- Statistical description of numeric columns (mean/std/min/max)

In [10]:
numeric_cols = ["duration_seconds","duration_min","season","episode","hd"]
numeric_cols = [c for c in numeric_cols if c in df_clean.columns]

desc = df_clean[numeric_cols].agg(["mean","std","min","max"]).T.round(3)
desc


,mean,std,min,max
duration_seconds,1350.177,7168.430,2.00,1461329.00
duration_min,22.503,119.474,0.03,24355.48
season,1.884,1.918,1.00,23.00
episode,10.596,11.323,1.00,282.00
hd,0.338,0.473,0.00,1.00


In [11]:
quantiles = df_clean[numeric_cols].quantile([0.25,0.5,0.75]).T.rename(columns={0.25:"Q1",0.5:"Median",0.75:"Q3"}).round(3)
quantiles


,Q1,Median,Q3
duration_seconds,43.0,249.0,1361.0
duration_min,0.72,4.15,22.68
season,1.0,1.0,2.0
episode,7.0,9.0,10.0
hd,0.0,0.0,1.0


7- Quick summary of the content

In [12]:
if "program_class" in df_clean.columns:
    print("Program Class counts:")
    print(df_clean["program_class"].value_counts(dropna=False).head(10))

if "program_genre" in df_clean.columns:
    print("\nTop Genres:")
    print(df_clean["program_genre"].value_counts(dropna=False).head(10))

if "hd" in df_clean.columns:
    hd_share = (df_clean["hd"].mean() * 100).round(2)
    print(f"\nHD Share: {hd_share}%")


Program Class counts:
program_class
SERIES/EPISODES    554276
MOVIE              392225
Name: count, dtype: int64

Top Genres:
program_genre
Animation      358203
Action         162106
Drama          142173
Comedy         124800
Horror          68918
Thriller        37650
Family          25885
Biography       11272
Documentary      8450
Crime            2757
Name: count, dtype: int64

HD Share: 33.83%
